In [4]:
import os
import vrplib
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
files = np.unique(['P/'+x[:-4] for x in os.listdir('P')])
files = np.concatenate((files, np.unique(['B/'+x[:-4] for x in os.listdir('B')])))
files = np.concatenate((files, np.unique(['E/'+x[:-4] for x in os.listdir('E')])))

In [34]:
import vrplib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
font = {
    'family': 'arial',
    'weight': 'bold',
    'size': 15
}
plt.rc('font', **font)
plt.rcParams["figure.figsize"] = (10, 8)


class ACOUnifiedVRP:
    """
    Объединённая версия алгоритма муравьиной колонии для VRP,
    использующая библиотеку vrplib и гиперпараметры/обновления феромонов
    из 'версии 2'.
    """

    def __init__(self,
                 alpha=0.1,      # Коэффициент для локального/глобального обновления феромонов
                 beta=0.95,      # Коэффициент влияния расстояния (жадность по расстоянию)
                 q0=1.0,         # Порог для жёсткого выбора (детерминированно vs рандомайзер)
                 epochs=4,       # Количество эпох (итераций)
                 num_ants=20,    # Количество муравьёв
                 W=1):           # Добавочная константа для расчёта L_nn
        self.alpha = alpha
        self.beta = beta
        self.q0 = q0
        self.epochs = epochs
        self.num_ants = num_ants
        self.W = W

        self.best_cost = float('inf')
        self.best_route = None

        self.cost = None
        self.routes = None
        self.dist_matrix = None
        self.demand = None
        self.capacity = None
        self.n_customers = None
        self.feromones = None
        self.tau_0 = None

    def fit(self, vrp_file_prefix, verbose=False):
        instance = vrplib.read_instance(f"{vrp_file_prefix}.vrp")
        solution = vrplib.read_solution(f"{vrp_file_prefix}.sol")

        self.cost = solution["cost"]
        self.routes = solution["routes"]

        dimension = instance["dimension"]  
        self.dist_matrix = np.array(instance["edge_weight"])
        self.demand = instance["demand"] 
        self.capacity = instance["capacity"]
        self.n_customers = dimension - 1
        
        min_dist = np.inf
        for i in range(dimension):
            for j in range(dimension):
                d = self.dist_matrix[i, j]
                if 0 < d < min_dist:
                    min_dist = d

        L_nn = (self.n_customers + 1) * min_dist + self.W
        self.tau_0 = 1.0 / L_nn
        self.feromones = np.ones((dimension, dimension)) * self.tau_0

        for epoch in range(self.epochs):
            for _ in range(self.num_ants):
                route, route_cost = self.construct_ant_solution()

                self.local_update(route)

                if route_cost < self.best_cost:
                    self.best_cost = route_cost
                    self.best_route = route

            self.global_update(self.best_route)

            # if verbose:
                # print(f"Epoch {epoch+1}/{self.epochs}, best cost so far: {self.best_cost:.2f}")

    def construct_ant_solution(self):
        route = [0]
        visited = set([0])
        total_cost = 0.0
        curr_capacity = self.capacity
        dimension = self.dist_matrix.shape[0]

        while len(visited) < dimension:
            current_node = route[-1]

            if current_node == 0:
                curr_capacity = self.capacity

            distances = self.dist_matrix[current_node]      
            pher = self.feromones[current_node]             

            attraction = np.zeros(dimension, dtype=float)
            for j in range(dimension):
                d = distances[j]
                if d > 0:
                    attraction[j] = (1 / d) ** self.beta
                else:
                    attraction[j] = 0

            for v in visited:
                attraction[v] = 0.0

            combined = pher * attraction
            sum_combined = combined.sum()

            if sum_combined == 0:
                if current_node != 0:
                    next_node = 0
                else:
                    break
            else:
                probs = combined / sum_combined

                if np.random.rand() < self.q0:
                    next_node = np.argmax(probs)
                else:
                    next_node = np.random.choice(range(dimension), p=probs)

            if next_node != 0:
                if self.demand[next_node] > curr_capacity:
                    next_node = 0

            dist_move = self.dist_matrix[current_node, next_node]
            total_cost += dist_move
            route.append(next_node)

            if next_node != 0:
                visited.add(next_node)
                curr_capacity -= self.demand[next_node]

            if len(visited) == dimension:
                if route[-1] != 0:
                    route.append(0)
                    total_cost += self.dist_matrix[route[-2], 0]
                break

        if route[-1] != 0:
            route.append(0)
            total_cost += self.dist_matrix[route[-2], 0]

        return route, total_cost

    def local_update(self, route):
        for i in range(len(route) - 1):
            a = route[i]
            b = route[i+1]
            self.feromones[a, b] = (1 - self.alpha) * self.feromones[a, b] + self.alpha * self.tau_0

    def global_update(self, best_route):
        if not best_route or len(best_route) < 2 or self.best_cost == float('inf'):
            return

        for i in range(len(best_route) - 1):
            a = best_route[i]
            b = best_route[i+1]
            self.feromones[a, b] = (1 - self.alpha) * self.feromones[a, b] + \
                                   self.alpha * (1.0 / self.best_cost)

    def get_best_solution(self):
        return self.best_route, self.best_cost

    def evaluate_solution(self):
        if self.cost is None or self.cost == 0:
            return None
        improvement = self.best_cost - self.cost
        rel_error = improvement / self.cost
        return rel_error

    def plot_routes(self):
        """
        Вспомогательная функция для визуализации эталонного и найденного маршрута.
        Требует, чтобы в instance['coordinates'] были координаты каждой вершины.
        В некоторых форматах VRP они могут отсутствовать.
        """
        pass
        # coords = instance["coordinates"] 
        # if not coords:
        #     print("No coordinates in instance for plotting.")
        #     return
        #
        # routes = self.routes 
        # self._plot_one_solution(coords, routes, "Bench Solution")
        #
        # if self.best_route:
        #     self._plot_one_solution(coords, [self.best_route], "Our Best Solution")

    def _plot_one_solution(self, coords, routes, title="Solution"):
        plt.figure()
        plt.title(title)
        for r_idx, route in enumerate(routes):
            x_coords = [coords[v][0] for v in route]
            y_coords = [coords[v][1] for v in route]
            plt.plot(x_coords, y_coords, marker='o', label=f"Route {r_idx+1}")
        plt.legend()
        plt.show()


In [33]:
differences = []
for prefix in files:
    print(f"\nЗапуск муравьиного алгоритма для: {prefix}")

    aco = ACOUnifiedVRP(
        alpha=0.3,  
        beta=5, 
        q0=0.7,  
        epochs=50,  
        num_ants=20,
        W=20
    )

    aco.fit(prefix, verbose=True)

    best_route, best_cost = aco.get_best_solution()

    rel_error = aco.evaluate_solution()
    differences.append(rel_error)
    
    # print(f"Лучший маршрут:  {best_route}")
    print(f"Стоимость:       {best_cost:.2f}")
    if rel_error is not None:
        print(f"Отклонение от эталона: {rel_error * 100:.2f}%")

    aco.plot_routes()
print(np.mean(differences))


Запуск муравьиного алгоритма для: P/P-n101-k4
Стоимость:       814.54
Отклонение от эталона: 19.61%

Запуск муравьиного алгоритма для: P/P-n16-k8
Стоимость:       459.46
Отклонение от эталона: 2.10%

Запуск муравьиного алгоритма для: P/P-n19-k2
Стоимость:       228.87
Отклонение от эталона: 7.96%

Запуск муравьиного алгоритма для: P/P-n20-k2
Стоимость:       229.86
Отклонение от эталона: 6.42%

Запуск муравьиного алгоритма для: P/P-n21-k2
Стоимость:       222.79
Отклонение от эталона: 5.59%

Запуск муравьиного алгоритма для: P/P-n22-k2
Стоимость:       224.65
Отклонение от эталона: 4.00%

Запуск муравьиного алгоритма для: P/P-n22-k8
Стоимость:       631.89
Отклонение от эталона: 4.79%

Запуск муравьиного алгоритма для: P/P-n23-k8
Стоимость:       554.25
Отклонение от эталона: 4.77%

Запуск муравьиного алгоритма для: P/P-n40-k5
Стоимость:       530.06
Отклонение от эталона: 15.73%

Запуск муравьиного алгоритма для: P/P-n45-k5
Стоимость:       585.13
Отклонение от эталона: 14.73%

Запус

In [18]:
q0 = [0.3, 0.4, 0.5, 0.7, 0.9]
alpha = [0.3, 0.5, 0.6, 0.7, 0.8, 0.9]
beta = [1, 2]
epochs=[4, 10]
W = [1, 10, 100]
ants = [4, 10]

scores = dict()
for a in alpha:
    for b in beta:
        for q in q0:
            for e in epochs:
                for an in ants:
                    for w in W:
                        differences = []
                        for prefix in files:
                            aco = ACOUnifiedVRP(
                                alpha=a,  
                                beta=b, 
                                q0=q,  
                                epochs=e,  
                                num_ants=an,
                                W=w
                            )
                            aco.fit(prefix, verbose=True)
                            rel_error = aco.evaluate_solution()
                            differences.append(rel_error)
                        scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'] = np.mean(differences)
                        print(scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'], f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}')

0.6913978342973425 alpha=0.3 beta=1 q0=0.3 epochs=4 ants=4 W=1
0.6728508535289923 alpha=0.3 beta=1 q0=0.3 epochs=4 ants=4 W=10
0.6988594312991191 alpha=0.3 beta=1 q0=0.3 epochs=4 ants=4 W=100
0.6190470089724102 alpha=0.3 beta=1 q0=0.3 epochs=4 ants=10 W=1
0.62351349376214 alpha=0.3 beta=1 q0=0.3 epochs=4 ants=10 W=10
0.631167897025073 alpha=0.3 beta=1 q0=0.3 epochs=4 ants=10 W=100
0.6403575799496554 alpha=0.3 beta=1 q0=0.3 epochs=10 ants=4 W=1
0.6362059828050127 alpha=0.3 beta=1 q0=0.3 epochs=10 ants=4 W=10
0.6274105275260415 alpha=0.3 beta=1 q0=0.3 epochs=10 ants=4 W=100
0.5770050499491692 alpha=0.3 beta=1 q0=0.3 epochs=10 ants=10 W=1
0.5665947481990811 alpha=0.3 beta=1 q0=0.3 epochs=10 ants=10 W=10
0.5821026780591121 alpha=0.3 beta=1 q0=0.3 epochs=10 ants=10 W=100
0.5942000610669386 alpha=0.3 beta=1 q0=0.4 epochs=4 ants=4 W=1
0.5998789808348508 alpha=0.3 beta=1 q0=0.4 epochs=4 ants=4 W=10
0.6000917549918543 alpha=0.3 beta=1 q0=0.4 epochs=4 ants=4 W=100
0.5302699480909445 alpha=0.3 be

In [19]:
min_1 = np.min(list(scores.values()))
sc1 = list(scores.keys())[np.argmin(list(scores.values()))]
min_1, sc1

(np.float64(0.18435543032666538),
 'alpha=0.3 beta=2 q0=0.9 epochs=10 ants=10 W=1')

In [20]:
q0 = [0.9, 1]
alpha = [0.1, 0.2, 0.3]
beta = [2]
epochs=[10]
W = [1, 10, 100]
ants = [10]

scores = dict()
for a in alpha:
    for b in beta:
        for q in q0:
            for e in epochs:
                for an in ants:
                    for w in W:
                        differences = []
                        for prefix in files:
                            aco = ACOUnifiedVRP(
                                alpha=a,  
                                beta=b, 
                                q0=q,  
                                epochs=e,  
                                num_ants=an,
                                W=w
                            )
                            aco.fit(prefix, verbose=True)
                            rel_error = aco.evaluate_solution()
                            differences.append(rel_error)
                        scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'] = np.mean(differences)
                        print(scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'], f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}')

0.1832183234049641 alpha=0.1 beta=2 q0=0.9 epochs=10 ants=10 W=1
0.18257535140266637 alpha=0.1 beta=2 q0=0.9 epochs=10 ants=10 W=10
0.1831155163623455 alpha=0.1 beta=2 q0=0.9 epochs=10 ants=10 W=100
0.23772462959789548 alpha=0.1 beta=2 q0=1 epochs=10 ants=10 W=1
0.24109678414680485 alpha=0.1 beta=2 q0=1 epochs=10 ants=10 W=10
0.24433504571794548 alpha=0.1 beta=2 q0=1 epochs=10 ants=10 W=100
0.18539168509618348 alpha=0.2 beta=2 q0=0.9 epochs=10 ants=10 W=1
0.17896948007944286 alpha=0.2 beta=2 q0=0.9 epochs=10 ants=10 W=10
0.1841604715697144 alpha=0.2 beta=2 q0=0.9 epochs=10 ants=10 W=100
0.2399340574976667 alpha=0.2 beta=2 q0=1 epochs=10 ants=10 W=1
0.24040021002198608 alpha=0.2 beta=2 q0=1 epochs=10 ants=10 W=10
0.24492226049241925 alpha=0.2 beta=2 q0=1 epochs=10 ants=10 W=100
0.1875048344060148 alpha=0.3 beta=2 q0=0.9 epochs=10 ants=10 W=1
0.18281801233711562 alpha=0.3 beta=2 q0=0.9 epochs=10 ants=10 W=10
0.18368580327317247 alpha=0.3 beta=2 q0=0.9 epochs=10 ants=10 W=100
0.2509188908

In [21]:
min_1 = np.min(list(scores.values()))
sc1 = list(scores.keys())[np.argmin(list(scores.values()))]
min_1, sc1

(np.float64(0.17896948007944286),
 'alpha=0.2 beta=2 q0=0.9 epochs=10 ants=10 W=10')

In [22]:
q0 = [0.9]
alpha = [0.1, 0.2]
beta = [2, 3, 5]
epochs=[10]
W = [1, 10, 100]
ants = [10]

scores = dict()
for a in alpha:
    for b in beta:
        for q in q0:
            for e in epochs:
                for an in ants:
                    for w in W:
                        differences = []
                        for prefix in files:
                            aco = ACOUnifiedVRP(
                                alpha=a,  
                                beta=b, 
                                q0=q,  
                                epochs=e,  
                                num_ants=an,
                                W=w
                            )
                            aco.fit(prefix, verbose=True)
                            rel_error = aco.evaluate_solution()
                            differences.append(rel_error)
                        scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'] = np.mean(differences)
                        print(scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'], f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}')

0.1820232588534395 alpha=0.1 beta=2 q0=0.9 epochs=10 ants=10 W=1
0.18298890454569355 alpha=0.1 beta=2 q0=0.9 epochs=10 ants=10 W=10
0.18466183657259602 alpha=0.1 beta=2 q0=0.9 epochs=10 ants=10 W=100
0.17707779655253036 alpha=0.1 beta=3 q0=0.9 epochs=10 ants=10 W=1
0.1751714947169089 alpha=0.1 beta=3 q0=0.9 epochs=10 ants=10 W=10
0.17404218511527056 alpha=0.1 beta=3 q0=0.9 epochs=10 ants=10 W=100
0.17381611454750162 alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=1
0.17918160173791245 alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=10
0.1800357217153351 alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=100
0.18709089039059554 alpha=0.2 beta=2 q0=0.9 epochs=10 ants=10 W=1
0.18484047551740537 alpha=0.2 beta=2 q0=0.9 epochs=10 ants=10 W=10
0.18467560912896946 alpha=0.2 beta=2 q0=0.9 epochs=10 ants=10 W=100
0.18212710533290116 alpha=0.2 beta=3 q0=0.9 epochs=10 ants=10 W=1
0.18470970539224077 alpha=0.2 beta=3 q0=0.9 epochs=10 ants=10 W=10
0.18117649651784568 alpha=0.2 beta=3 q0=0.9 epochs=10 ants=10 W=10

In [23]:
min_1 = np.min(list(scores.values()))
sc1 = list(scores.keys())[np.argmin(list(scores.values()))]
min_1, sc1

(np.float64(0.17381611454750162),
 'alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=1')

In [26]:
q0 = [0.9]
alpha = [0.1]
beta = [4, 5]
epochs=[10]
W = [1, 10, 100]
ants = [10]

scores = dict()
for a in alpha:
    for b in beta:
        for q in q0:
            for e in epochs:
                for an in ants:
                    for w in W:
                        differences = []
                        for prefix in files:
                            aco = ACOUnifiedVRP(
                                alpha=a,  
                                beta=b, 
                                q0=q,  
                                epochs=e,  
                                num_ants=an,
                                W=w
                            )
                            aco.fit(prefix, verbose=True)
                            rel_error = aco.evaluate_solution()
                            differences.append(rel_error)
                        scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'] = np.mean(differences)
                        print(scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'], f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}')

0.1785535030329675 alpha=0.1 beta=4 q0=0.9 epochs=10 ants=10 W=1
0.17993578298979793 alpha=0.1 beta=4 q0=0.9 epochs=10 ants=10 W=10
0.17731435858993863 alpha=0.1 beta=4 q0=0.9 epochs=10 ants=10 W=100
0.180383549052264 alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=1
0.17626027559833973 alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=10
0.17367271143770638 alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=100


In [27]:
min_1 = np.min(list(scores.values()))
sc1 = list(scores.keys())[np.argmin(list(scores.values()))]
min_1, sc1

(np.float64(0.17367271143770638),
 'alpha=0.1 beta=5 q0=0.9 epochs=10 ants=10 W=100')

In [30]:
q0 = [0.7]
alpha = [0.1, 0.3]
beta = [2, 5]
epochs=[50]
W = [0, 10, 20]
ants = [20]

scores = dict()
for a in alpha:
    for b in beta:
        for q in q0:
            for e in epochs:
                for an in ants:
                    for w in W:
                        differences = []
                        for prefix in files:
                            aco = ACOUnifiedVRP(
                                alpha=a,  
                                beta=b, 
                                q0=q,  
                                epochs=e,  
                                num_ants=an,
                                W=w
                            )
                            aco.fit(prefix, verbose=True)
                            rel_error = aco.evaluate_solution()
                            differences.append(rel_error)
                        scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'] = np.mean(differences)
                        print(scores[f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}'], f'alpha={a} beta={b} q0={q} epochs={e} ants={an} W={w}')

0.17629100083211868 alpha=0.1 beta=2 q0=0.7 epochs=50 ants=20 W=0
0.18036162588450325 alpha=0.1 beta=2 q0=0.7 epochs=50 ants=20 W=10
0.17842862603364099 alpha=0.1 beta=2 q0=0.7 epochs=50 ants=20 W=20
0.14353804564846656 alpha=0.1 beta=5 q0=0.7 epochs=50 ants=20 W=0
0.14492091388718442 alpha=0.1 beta=5 q0=0.7 epochs=50 ants=20 W=10
0.14238095206880572 alpha=0.1 beta=5 q0=0.7 epochs=50 ants=20 W=20
0.18148133431116104 alpha=0.3 beta=2 q0=0.7 epochs=50 ants=20 W=0
0.1777807730889296 alpha=0.3 beta=2 q0=0.7 epochs=50 ants=20 W=10
0.1810372703254334 alpha=0.3 beta=2 q0=0.7 epochs=50 ants=20 W=20
0.14375781557455067 alpha=0.3 beta=5 q0=0.7 epochs=50 ants=20 W=0
0.1456386049256843 alpha=0.3 beta=5 q0=0.7 epochs=50 ants=20 W=10
0.13820111981918287 alpha=0.3 beta=5 q0=0.7 epochs=50 ants=20 W=20


In [31]:
min_1 = np.min(list(scores.values()))
sc1 = list(scores.keys())[np.argmin(list(scores.values()))]
min_1, sc1

(np.float64(0.13820111981918287),
 'alpha=0.3 beta=5 q0=0.7 epochs=50 ants=20 W=20')